# **Fabrica de relojes**

Vamos a construir una fábrica de relojes. El caja es construida por acero, el cristal por vidrio y las correas de piel. Después de eso, se combinan las partes fisicas con los componentes electrónicos, así se hace el reloj.



1.   Hay 4 contenedores principales. Acero, Vidrio, Piel y Electrónica. Esos contenedores tienen una cantidad N de componentes que se utilizarán en el proceso.
2.   La caja debe tomar una pieza de acero y transformarlo en una caja, que se almacenará en el almacenamiento de cajas. Lo mismo sucede con el constructor de correas, pero de una pieza de cuero obtendrá dos correas. se almacenara en el almacen de correasa y asi sucesivamente
3. El pintor elige cajas, los pinta y los almacena en el Almacenamiento de cajas 2
4. El ensamblador elige una caja, un vidrio, dos correas y un componente electronico, y ensambla el reloj, que se almacenará en el contenedor de despacho.
5. Después de hacer una cantidad N de relojes, envíe a alguien a la tienda a recogerlas.
6. Cuando la materia prima este por debajo de cierto nivel, se realiza una llamada al Proveedor. Después de T días, el proveedor llega a la fábrica y rellena el contenedor con materia prima.


In [ ]:
!pip install simpy

import simpy
import random

Agregamos los 4 primeros contenedores. Por supuesto, el envío comenzará vacío. Si nos quedamos sin alguna materia prima, el proceso no se detendrá, pero tampoco creará más guitarras. Lo mismo sucede si el despacho está lleno.

In [ ]:
relojes_fabricados = 0

#Parámetros

#Horas Laborales
horas = 8

#Dias hábiles
dias = 23

#tiempo total de trabajo (horas)
tiempo_total = horas * dias

#contenedores
     #acero
capacidad_acero = 500
inicial_acero = 200

     #electronico
capacidad_electronica = 100
inicial_electronica = 60

     #piel
capacidad_piel = 250
inicial_piel = 100

    #vidrio
capacidad_vidrio = 500
inicial_vidrio = 200

capacidad_despacho = 500

Ahora definiremos cuantos empleados y el tiempo que les tomara fabricar cierto componente, las actividades toman diferentes tiempos, incluso se repiten a lo largo del tiempo. Aquí asumiremos que la distribución del tiempo es normal

In [ ]:
#empleados por actividad
     #caja
caja_num = 2
caja_media = 1
caja_desv = 0.1

     #cristal
cristal_num = 1
cristal_media = 1
cristal_desv = 0.2

    #correas
correas_num = 2
correas_media = 1
correas_desv = 0.2

     #ensamblaje
ensam_num = 3
ensam_media = 3
ensam_desv = 0.3

Luego, para el nivel crítico que defino depende del tiempo medio que se tarda en crear cada componente, la cantidad de fabricantes de cada unos, los 2 días hábiles que tarda en llegar el proveedor y 1 día de margen solo para estar seguro

In [ ]:
#niveles críticos
     #El stock crítico debe ser 1 día hábil más de lo que tarda el proveedor en llegar

acero_critical_stock = ((8/caja_media) * caja_num)*3

vidrio_critical_stock = ((8/cristal_media) * cristal_num)*3

piel_critical_stock = ((8/correas_media) * correas_num)*3

elect_critical_stock = ((8/ensam_media) * ensam_num)*2

  1. El primer rendimiento env.timeout(0) significa que este proceso comenzará a ejecutarse tan pronto como comience la simulación. Podemos cambiar ese valor (0) a, por ejemplo, un 8. De esa manera, el proceso comenzará 8 unidades de tiempo después.

2. El while True, como explicamos anteriormente, significa que el proceso se ejecutará durante todo el tiempo que dure la simulación.

3. Luego, verificaría si el nivel de existencias es igual o menor que el nivel crítico definido anteriormente. Si el stock es mayor que ese nivel, se cerraría por 1 unidad de tiempo (vea la declaración else).

4. Si el nivel de stock es igual o inferior al nivel crítico, se ejecutará la impresión, informando que el nivel es N en un determinado día y hora y se ha realizado la llamada al proveedor.

5. Después de 2 días (16 horas), llega el proveedor y volvería a llenar el contenedor de acero con 300 unidades. Eso es lo que significa yield self.wood.put(300).

6. Finalmente, se imprime el nuevo nivel de existencias y la alarma se apagará durante 1 día (yield env.timeout(8)).

In [ ]:
class Fabrica_relojes:
   def __inicial__(self, env):
    self.acero = simpy.Container(env, capacity = capacidad_acero, init = inicial_acero)
    self.acero_control = env.process(self.acero_stock_control(env))

    self.piel = simpy.Container(env, capacity = capacidad_piel, init = inicial_piel)
    self.piel_control = env.process(self.piel_stock_control(env))

    self.vidrio = simpy.Container(env, capacity = capacidad_vidrio, init = inicial_vidrio)
    self.vidrio_control = env.process(self.vidrio_stock_control(env))

    self.electronica = simpy.Container(env, capacity = capacidad_electronica, init = inicial_electronica)
    self.electronica_control = env.process(self.electronica_stock_control(env))

    self.despacho = simpy.Container(env ,capacity = capacidad_despacho, init = 0)
    self.despacho_control = env.process(self.despacho_control(env))

    def acero_stock_control(self, env):
      yield env.timeout(0)
      while True:
          if self.acero.level <= acero_critical_stock:
              print('Existencias de acero por debajo del nivel crítico ({0}) en el día {1}, hora {2}'.format(
                  self.acero.level, int(env.now/8), env.now % 8))
              print('Llamando al proveedor de acero')
              print('----------------------------------')
              yield env.timeout(16)
              print('Proveedor de acero llega el día {0}, hora {1}'.format(
                  int(env.now/8), env.now % 8))
              yield self.acero.put(300)
              print('Nuevo stock de acero es {0}'.format(
                  self.acero.level))
              print('----------------------------------')
              yield env.timeout(8)
          else:
              yield env.timeout(1)

    def vidrio_stock_control(self, env):
      yield env.timeout(0)
      while True:
          if self.vidrio.level <= vidrio_critical_stock:
              print('Existencias de vidrio por debajo del nivel crítico ({0}) en el día {1}, hora {2}'.format(
                  self.vidrio.level, int(env.now/8), env.now % 8))
              print('Llamando al proveedor de vidrio')
              print('----------------------------------')
              yield env.timeout(16)
              print('Proveedor de vidrio llega el día {0}, hora {1}'.format(
                  int(env.now/8), env.now % 8))
              yield self.vidrio.put(300)
              print('Nuevo stock de vidrio es {0}'.format(
                  self.vidrio.level))
              print('----------------------------------')
              yield env.timeout(8)
          else:
              yield env.timeout(1)

    def piel_stock_control(self, env):
      yield env.timeout(0)
      while True:
          if self.piel.level <= piel_critical_stock:
              print('Existencias de piel por debajo del nivel crítico ({0}) en el día {1}, hora {2}'.format(
                  self.piel.level, int(env.now/8), env.now % 8))
              print('Llamando al proveedor de piel')
              print('----------------------------------')
              yield env.timeout(16)
              print('Proveedor de piel llega el día {0}, hora {1}'.format(
                  int(env.now/8), env.now % 8))
              yield self.piel.put(150)
              print('Nuevo stock de piel es {0}'.format(
                  self.piel.level))
              print('----------------------------------')
              yield env.timeout(8)
          else:
              yield env.timeout(1)

    def electronica_stock_control(self, env):
      yield env.timeout(0)
      while True:
          if self.electronica.level <= elect_critical_stock:
              print('Existencias de disp. electronicos por debajo del nivel crítico ({0}) en el día {1}, hora {2}'.format(
                  self.electronica.level, int(env.now/8), env.now % 8))
              print('Llamando al proveedor de dispositivos')
              print('----------------------------------')
              yield env.timeout(9)
              print('Proveedor de dispositivos llega el día {0}, hora {1}'.format(
                  int(env.now/8), env.now % 8))
              yield self.electronica.put(40)
              print('Nuevo stock de vidrio es {0}'.format(
                  self.electronica.level))
              print('----------------------------------')
              yield env.timeout(8)
          else:
              yield env.timeout(1)

    def despacho_control(self, env):
        global relojes_fabricados
        yield env.timeout(0)
        while True:
            if self.despacho.level >= 50:
                print('el stock de despacho es {0}, llamando a la tienda para recoger relojes el día {1}, hora {2}'.format(
                    self.dispatch.level, int(env.now/8), env.now % 8))
                print('----------------------------------')
                yield env.timeout(4)
                print('recoger en la tienda {0} relojes el día {1}, hora {2}'.format(
                    self.dispatch.level, int(env.now/8), env.now % 8))
                relojes_fabricados += self.despacho.level
                yield self.despacho.get(self.despacho.level)
                print('----------------------------------')
                yield env.timeout(8)
            else:
                yield env.timeout(1)

In [ ]:
def caja_empl(env, relojes):
    while True:
        yield relojes.acero.get(1)
        caja_tiempo = random.gauss(caja_media, caja_desv)
        yield env.timeout(caja_tiempo)
        yield relojes.almacen_caja.put(1)

def correas_empl(env, relojes):
    while True:
        yield relojes.piel.get(1)
        correas_tiempo = random.gauss(correas_media, correas_desv)
        yield env.timeout(correas_tiempo)
        yield relojes.almacen_correas.put(2)

def cristal_empl(env, relojes):
    while True:
        yield relojes.vidrio.get(1)
        cristal_tiempo = random.gauss(cristal_media, cristal_desv)
        yield env.timeout(cristal_tiempo)
        yield relojes.almacen_cristal.put(1)

def ensamblador(env, relojes):
    while True:
        yield relojes.almacen_caja.get(1)
        yield relojes.almacen_correas.get(2)
        yield relojes.almacen_cristal.get(1)
        yield relojes.almacen_electronica.get(1)
        tiempo_ensamblado = max(random.gauss(ensam_media, ensam_desv), 1)
        yield env.timeout(tiempo_ensamblado)
        yield relojes.despacho.put(1)


Empezamos a crear nuestros dos primeros empleados. La función toma dos argumentos: el entorno SimPy y la clase Fabrica_reljes:

1. Mientras se ejecuta la simulación, el fabricante tomará una pieza de acero (yield fabrica_relojes.acero.get(1)).

2. Mantendrá esa pieza de madera por un período de tiempo (eso es lo que hace la función env.timeout). Esto simula el tiempo que lleva transformar una pieza de acero en una caja.

3. Después de que haya pasado esa unidad de tiempo (1 en nuestro caso), el fabricante colocará esa pieza en un contenedor llamado despacho.





In [ ]:
#Generadores

def fab_caja(env, relojes):
    for i in range(caja_num):
        env.process(fab_caja(env, relojes))
        yield env.timeout(0)

def fab_cristal(env, relojes):
    for i in range(cristal_num):
        env.process(fab_cristal(env, relojes))
        yield env.timeout(0)

def fab_correas(env, relojes):
    for i in range(correas_num):
        env.process(fab_correas(env, relojes))
        yield env.timeout(0)

def fab_ensam(env, relojes):
    for i in range(ensam_num):
        env.process(fab_ensam(env, relojes))
        yield env.timeout(0)


In [ ]:
env = simpy.Environment()
relojes = Fabrica_relojes

In [ ]:
caja_gen = env.process(fab_caja(env, relojes))
cristal_gen = env.process(fab_cristal(env, relojes))
correas_gen = env.process(fab_correas(env, relojes))
ensamblado_gen = env.process(fab_ensam(env, relojes))

In [ ]:
env.run(until = tiempo_total)

In [ ]:
print(f'¡El despacho tiene %d relojes listas para usar!' % relojes.despacho.level)
print(f'----------------------------------')
print('Total relojes fabricados: {0}'.format(relojes_fabricados + relojes.despacho.level))
print(f'----------------------------------')
print(f'SIMULATION COMPLETED')